In [8]:
import pandas as pd
import random
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import operator
import itertools
import torch

ERROR! Session/line number was not unique in database. History logging moved to new session 251


In [9]:
# Функция для вычисления расстояния между двумя городами
def distance(x1, y1, x2, y2):
    res = np.sqrt((x1 - x2)**2 + (y1 - y2)**2)
    return res

# Функция для вычисление длины всего тура
def totaldistancetour(tour, cities, depot):
    d = 0
    for i in range(1, len(tour)):
        x1 = cities[int(tour[i - 1])][0]
        y1 = cities[int(tour[i - 1])][1]
        x2 = cities[int(tour[i])][0]
        y2 = cities[int(tour[i])][1]
        d = d + distance(x1, y1, x2, y2)
    
    x1 = cities[int(tour[len(tour) - 1])][0]
    y1 = cities[int(tour[len(tour) - 1])][1]
    x2 = depot[0][0]
    y2 = depot[0][1]
    d = d + distance(x1, y1, x2, y2)
    
    x1 = cities[int(tour[0])][0]
    y1 = cities[int(tour[0])][1]
    x2 = depot[0][0]
    y2 = depot[0][1]
    d = d + distance(x1, y1, x2, y2)
    return d

def subtourslice(tour, vehicle, max_vehicle):
    #print("--------------------------------Начало работы функции--------------------------------")
    capacity_used = np.zeros(len(vehicle))
    k = 0
    slice = []
    mass = [] # список всех весов грузов для перевозки
    for i in range(len(vehicle)):
        # capacity_used[i] - начальный вес
        # vehicle[i][1] - это значение из колонки 8000, то есть максимальный объем
        # vehicle[i][0] - это первая колонка, то есть номер грузовика
        #print("Начальный суммарный вес: ", capacity_used[i])
        while((capacity_used[i] <= max_vehicle) and (k <= (len(tour) - 1))):
            # cities[tour[k]][2] - кол-во груза, который грузовик везет в город k (третья колнка в первой таблице)
            # print("Номер города для входа: ", k)
            # print("Временный суммарный вес: ", capacity_used[i])
            # print("Вес груза, который нужно перевезти в текущий город: ", cities[tour[k]][2])
            capacity_used[i] += vehicle[int(tour[k])] # заполняем грузовик конкретным кол-вом груза для каждого города
            if(capacity_used[i] > max_vehicle):
                # если кол-во груза, который нужно отвезти, больше чем вместимость грузовика, то вычитаем это кол-во груза
                capacity_used[i] -= vehicle[int(tour[k])]
                k -= 1 # это номер города после которого суммарный вес > объема грузовика и дальнейший вес груза не учитываем
                # print("Номер города, после которого суммарный вес > объема грузовика: ", k)
                # после такого города, нужно полностью очистить грузовик и дальше отправить его по городам
                slice.append(k) # запоминаем номер тура, чтобы повторно не проходить по такому же маршруту
                k += 1
                break
            k += 1
            #print(' ')
        #print(' ')
        mass.append(capacity_used[i])
    slice.append(k - 1)
    return(slice, mass)

# Что происходит в цикле while
# Мы на каждом шаге суммарный вес груза увеличиваем для каждого грузовика пока он не превысит максимальный объем вмесимости
# грузовика. Если это происходит, то мы запоминаем предыдущий номер города после которого общий вес превысиль обьем грузовика,
# возвращаемся к номеру города, на котором остановились и начинаем все заново.

# print(subtourslice(tour, vehicle))

In [10]:
def subtour(slice, tour):
    sub = []
    # добавляем номера городов, после которых суммарный вес груза превышал объем грузовика
    sub.append(tour[:(slice[0] + 1)]) # создаем список от 0 до 8
    for i in range(0, len(slice) - 1):
        # нарежем маршруты городов до момента, пока суммарный вес груза не превысел объем грузовика
        sub.append(tour[(slice[i] + 1):(slice[i + 1] + 1)])
        # создает список от 9 до 15, потом от 16 до 20
    return sub

In [11]:
# Вычисление общей длины всех туров

def all_vechile_distance(sub, cities, depot):
    all_distance = totaldistancetour(sub, cities, depot)
    return all_distance

In [12]:
# Функция определения общего расстояния по данным туров и грузовиков
def tour_to_distance(tour, cities, vehicle, max_vehicle, depot):
    u = subtourslice(tour, vehicle, max_vehicle) # получаем спислок городов, после которых суммарный вес груза > объема грузовика
    v = subtour(u[0], tour) # получим тур из номеров городов, где u[0] это список из номеров городов, после которых суммарный вес груза > объема грузовика
    total = all_vechile_distance(v[0], cities, depot) # посчитаем суммарное расстояние между всеми городами в туре
    return total

In [13]:
def Initialize(count): # задаем список из count городов и случайно их перемешиваем
    solution = np.arange(count)
    np.random.shuffle(solution)
    return solution

def GenerateStateCandidate(current):
    new = current.copy()
    index_a = np.random.randint(len(current))
    index_b = np.random.randint(len(current))
    while index_b == index_a:
        index_b = np.random.randint(len(current))
    if(index_a > index_b):
        new[index_b:index_a] = np.flip(new[index_b:index_a])
    else:
        new[index_a:index_b] = np.flip(new[index_a:index_b])
    return new

In [14]:
def SA(cities, vehicle, max_vehicle, T_end, t_max, depot):
    current_solution = Initialize(len(cities))
    currentEnergy = tour_to_distance(current_solution, cities, vehicle, max_vehicle, depot) # вычисляем энергию для первого состояния
    best_tour = np.copy(current_solution)
    T = t_max
    best_Energy = worst_Energy = currentEnergy
    while(T > T_end):  # на всякий случай ограничеваем количество итераций
    # может быть полезно при тестировании сложных функций изменения температуры T       
        #print("Подшаг_шага:", k)
        new_solution = GenerateStateCandidate(current_solution) # получаем новое решение
        candidateEnergy = tour_to_distance(current_solution, cities, vehicle, max_vehicle, depot) # вычисляем его энергию
        #print("Текущая энергия: ", candidateEnergy)
        best_Energy = min(best_Energy, candidateEnergy)
        worst_Energy = max(worst_Energy, candidateEnergy)

        if(candidateEnergy < currentEnergy): # если кандидат обладает меньшей энергией
            currentEnergy = candidateEnergy # то оно становится текущим состоянием
            current_solution = np.copy(new_solution)
            if(currentEnergy < best_Energy):
                best_tour = np.copy(current_solution)
        else:
            p = np.exp((currentEnergy - candidateEnergy) / T) # иначе, считаем вероятность
            if (p > np.random.uniform()): # и смотрим, осуществится ли переход
                currentEnergy = candidateEnergy
                best_tour = np. copy(current_solution)
                current_solution = np.copy(new_solution)
        T *= 0.9995 # уменьшаем температуру
    return best_Energy

In [17]:
%%time

# Для 20 вершин объем грузовика 500, для 50 вершин объем грузовика 750, для 50 вершин объем грузовика 1000

n_samples = 2000
n_customer = 20

T_end = 1
t_max = 100000

rnds = np.random

# максимальная вместимость грузовика
max_vehicle = 500

all_results = []

i = 0
while(i < n_samples):
    # Cоздаем одно депо, n_customer координат городов и n_customer весов
    depot = rnds.uniform(size=(1, 2))  # depot location
    cities = rnds.uniform(size=(n_customer, 2))  # node locations
    vehicle = list(rnds.randint(1, 42, n_customer))
    print("Шаг:", i)
    res = SA(cities, vehicle, max_vehicle, T_end, t_max, depot)
    all_results.append(res)
    i += 1
print(all_results)

Шаг: 0
Шаг: 1
Шаг: 2
Шаг: 3
Шаг: 4
Шаг: 5
Шаг: 6
Шаг: 7
Шаг: 8
Шаг: 9
Шаг: 10
Шаг: 11
Шаг: 12
Шаг: 13
Шаг: 14
Шаг: 15
Шаг: 16
Шаг: 17
Шаг: 18
Шаг: 19
Шаг: 20
Шаг: 21
Шаг: 22
Шаг: 23
Шаг: 24
Шаг: 25
Шаг: 26
Шаг: 27
Шаг: 28
Шаг: 29
Шаг: 30
Шаг: 31
Шаг: 32
Шаг: 33
Шаг: 34
Шаг: 35
Шаг: 36
Шаг: 37
Шаг: 38
Шаг: 39
Шаг: 40
Шаг: 41
Шаг: 42
Шаг: 43
Шаг: 44
Шаг: 45
Шаг: 46
Шаг: 47
Шаг: 48
Шаг: 49
Шаг: 50
Шаг: 51
Шаг: 52
Шаг: 53
Шаг: 54
Шаг: 55
Шаг: 56
Шаг: 57
Шаг: 58
Шаг: 59
Шаг: 60
Шаг: 61
Шаг: 62
Шаг: 63
Шаг: 64
Шаг: 65
Шаг: 66
Шаг: 67
Шаг: 68
Шаг: 69
Шаг: 70
Шаг: 71
Шаг: 72
Шаг: 73
Шаг: 74
Шаг: 75
Шаг: 76
Шаг: 77
Шаг: 78
Шаг: 79
Шаг: 80
Шаг: 81
Шаг: 82
Шаг: 83
Шаг: 84
Шаг: 85
Шаг: 86
Шаг: 87
Шаг: 88
Шаг: 89
Шаг: 90
Шаг: 91
Шаг: 92
Шаг: 93
Шаг: 94
Шаг: 95
Шаг: 96
Шаг: 97
Шаг: 98
Шаг: 99
Шаг: 100
Шаг: 101
Шаг: 102
Шаг: 103
Шаг: 104
Шаг: 105
Шаг: 106
Шаг: 107
Шаг: 108
Шаг: 109
Шаг: 110
Шаг: 111
Шаг: 112
Шаг: 113
Шаг: 114
Шаг: 115
Шаг: 116
Шаг: 117
Шаг: 118
Шаг: 119
Шаг: 120
Шаг: 121
Шаг: 122
Шаг

In [18]:
print(np.mean(all_results))

7.20879027758146
